# La vectorisation, ou comment aider les machines à nous comprendre

Le changement de paradigme s’est opéré sans le voir, dans un certain enthousiasme pour le progrès technologique : alors que nous avions créé les machines pour nous alléger la tâche, nous voilà maintenant à les servir dans le même but.

Avec le développement du *machine learning* et ses prouesses que l’on vante quotidiennement dans la presse, les réseaux autorisés ou encore dans les cercles d’amis, la notion d’entraînement supervisé ou non-supervisé nous est devenue familière : pour qu’un algorithme d’apprentissage fonctionne bien, il faut le nourrir avec une quantité proverbiale de données.

Comment, alors, procéder ? Pour qu’une machine comprenne le sens d’un texte, suffit-il de lui dire qu’elle peut trouver ici un verbe conjugué, là le sujet, et que tout le reste sert à définir des circonstances ? Comment parvenir à lui dire que *un chat* et *un miaou* désignent la même réalité, et en même temps pas tout à fait ?

Le procédé repose sur la nécessité de présenter à la machine un objet sous forme numérique. L’idée n’est pas nouvelle. Déjà au IVe siècle avant J.-C, l’inscription « Que nul n’entre ici s’il n’est géomètre » que Platon aurait fait graver au fronton de l’Académie d’Athènes, supposait l’ambition de faire la synthèse entre le monde sensible et celui des idées pures. Plus tard, Galilée affirmait que « la nature est un livre écrit en langage mathématique ». Que dire aussi de la méthode de Descartes ou encore de l’*Éthique* de Spinoza, qui emprunte sa forme à la déduction mathématique ?

Et c’est bien un objet issu des mathématiques, le vecteur, qui sert aujourd’hui de support à la traduction des mots dans le langage des machines. Il a cela de commode qu’il se laisse facilement manipuler par des opérations pour le comparer à d’autres vecteurs, autorisant de répondre à une question aussi essentielle que par exemple calculer la distance entre le vecteur $chat$ et le vecteur $miaou$.

Les outils mathématiques existant déjà, tout l’art est celui de l’alchimiste : découvrir le nombre secret de l’objet.

## Une approche naïve

Quel nombre attribuer à un texte ou, plus spécifiquement, à son composant premier et emblématique, le mot ?

Considérons deux phrases qui forment notre corpus :

```txt
(a) Le petit chat boit du lait.
(b) Le petit chien boit de l’eau.
```

Après la phase de segmentation en mots, elles peuvent se représenter dans deux vecteurs $\vec{A}$  et $\vec{B}$ :

$$
\vec{A} = \begin{pmatrix}
    \text{le} \\
    \text{petit} \\
    \text{chat} \\
    \vdots \\
    \text{lait}
\end{pmatrix}
\hspace{2em}
\vec{B} = \begin{pmatrix}
    \text{le} \\
    \text{petit} \\
    \text{chien} \\
    \vdots \\
    \text{eau}
\end{pmatrix}
$$

De cette opération, il devient facile de constituer le vocabulaire de notre corpus dans une suite de mots :

$$
S = \text{boit}, \text{chat}, \text{chien}, \text{de}, \text{du}, \text{eau}, \text{lait}, \text{l}, \text{le}, \text{petit}
$$

De telle manière que :

$$
S_0 = \text{boit}
$$

Une piste possible serait d’attribuer à chaque élément des vecteurs, les mots, leur numéro d’ordre dans le vocabulaire :

$$
\vec{A} = \begin{pmatrix}
    S_8 \\
    S_9 \\
    S_1 \\
    \vdots \\
    S_6
\end{pmatrix}
\hspace{2em}
\vec{B} = \begin{pmatrix}
    S_8 \\
    S_9 \\
    S_2 \\
    \vdots \\
    S_5
\end{pmatrix}
$$

Puis, calculer la distance qui les sépare en faisant la somme de leurs différences ou, en langage mathématique, définir **la norme du vecteur** :

$$
\| \vec{AB} \| = \sqrt{(A_1 - B_1)^2 + (A_2 - B_2)^2 \dots (A_n - B_n)^2}
$$

En Python, nous pourrions formaliser les instructions suivantes :

In [ ]:
def distance(*, a:list, b:list) -> float:
    """Euclidean distance between two vectors.
    
    Keyword arguments:
    a -- first vector
    b -- second vector
    """
    # difference between indices
    coords = [
        (x - y) ** 2
        for x, y in zip(a, b)
    ]
    # distance = square root of the sum of coords
    return sum(coords) ** .5

# sentences into tokens
a = ["le", "petit", "chat", "boit", "du", "lait"]
b = ["le", "petit", "chien", "boit", "de", "l", "eau"]

# corpus vocabulary
S = ["boit", "chat", "chien", "de", "du", "eau", "lait", "l", "le", "petit"]

# indices of elements rather than values
A = [ S.index(x) for x in a ]
B = [ S.index(y) for y in b ]

print(f"La distance entre les deux phrases est de {distance(a=A, b=B) :.4f}")

## L’approche fréquentielle

En vérité, les opérations réalisées n’ont pas beaucoup de sens. Dire qu’il existe « une distance de 1.7321 », ça signifie quoi exactement ? Ça semble peu, mais par rapport à quoi ? Quelle est l’unité de référence ? Tout ça sans oublier que, en plus, les vecteurs ne sont pas de tailles égales : une phrase qui ne comporterait qu’un seul mot se révélerait extrêmement proche de toute autre phrase qui commencerait par ce même mot. Nous pourrions refaire le calcul de distance en supprimant le dernier élément de `B` afin d’obtenir des vecteurs de tailles égales que nous obtiendrions le même résultat :

In [ ]:
print(f"La distance entre les deux phrases est toujours de {distance(a=A, b=B[:-1]) :.4f}")

Le nombre secret d’un mot ne semble donc pas être sa position dans un vocabulaire de référence. La première méthode véritable consiste à opter pour une approche fréquentielle : du corpus on obtient un sac de mots puis, pour chaque texte, on génère un vecteur de la longueur du corpus dont les dimensions sont le nombre d’occurrences de chaque mot.

Considérons les deux textes suivants :

```txt
(a) Le petit chat boit du lait. Le lait n’est pas bon pour les chats.
(b) Les petits chiens boivent de l’eau. L’eau irait aussi aux chats.
```

La construction d’un sac de mots (*bag of words*, BOW) implique d’autres méthodes que simplement la segmentation en mots. Il s’agit aussi de ne conserver que les mots signifiants et de les lemmatiser à partir, souvent, de leur étiquette morpho-syntaxique.

De notre corpus, nous obtiendrions le sac de mots suivant, constitué de 9 éléments :

$$
\text{BOW} = \text{aller}, \text{aussi}, \text{boire}, \text{bon}, \text{chat}, \text{chien}, \text{eau}, \text{lait}, \text{petit}
$$

L’étape suivante est de compter, pour chacun des textes, le nombre d’occurrences des éléments du sac de mots :

$$
\begin{align}
\vec{A} = \begin{pmatrix}
    \text{(aller :) } &0 \\
    \text{(aussi :) } &0 \\
    \text{(boire :) } &1 \\
    \text{(bon :) } &1 \\
    \text{(chat :) } &2 \\
    \text{(chien :) } &0 \\
    \text{(eau :) } &0 \\
    \text{(lait :) } &2 \\
    \text{(petit :) } &1 \\
\end{pmatrix}
\hspace{5em}
\vec{B} = \begin{pmatrix}
    \text{(aller :) } &1 \\
    \text{(aussi :) } &1 \\
    \text{(boire :) } &1 \\
    \text{(bon :) } &0 \\
    \text{(chat :) } &1 \\
    \text{(chien :) } &1 \\
    \text{(eau :) } &2 \\
    \text{(lait :) } &0 \\
    \text{(petit :) } &1 \\
\end{pmatrix}
\end{align}
$$

Les vecteurs étant de même longueur, il est cette fois-ci pertinent de normaliser le vecteur $\vec{AB}$ :

In [ ]:
A = [0, 0, 1, 1, 2, 0, 0, 2, 1]
B = [1, 1, 1, 0, 1, 1, 2, 0, 1]

print(f"La distance entre les deux phrases est de {distance(a=A, b=B) :.4f}")

Le résultat n’a pas forcément plus de sens que lors de notre approche naïve, mais il permettrait d'effectuer un classement de proximité entre plusieurs phrases.

## L’encodage 1 parmi *n*

Plus connue sous sa dénomination anglaise, *one-hot encoding*, et parfois appelée "encodage à chaud", cette méthode consiste à recueillir toutes les catégories possibles dans un vecteur et à les transformer en autant de classificateurs binaires ($0$ ou $1$).

Considérons deux phrases étiquetées en parties du discours :

```txt
(a) Le/DET petit/ADJ chat/N boit/V du/DET lait/N ./PONCT
(b) Le/DET petit/ADJ chien/N boit/V de/DET l’/DET eau/N ./PONCT
```

Après la phase de pré-traitement, nous obtenons le sac de mots ci-dessous :

$$
\text{BOW} = (\text{boire}, \text{V}), (\text{chat}, \text{N}), (\text{chien}, \text{N}), (\text{eau}, \text{N}), (\text{lait}, \text{N}), (\text{petit}, \text{ADJ})
$$

Notons bien que chaque élément de la série $BOW$ peut se matérialiser comme un vecteur de longueur 2 :

$$
\vec{BOW_1} = \begin{pmatrix}
    \text{boire} \\
    \text{V}
\end{pmatrix}
$$

L’encodage *one-hot* détermine tout d’abord, à partir du sac de mots, les catégories à transformer en classificateurs. Chaqué élément de la série $BOW$ étant de longueur 2, deux ensembles de catégories seront créés : la première pour les lemmes, la seconde pour les étiquettes.

$$
\begin{align}
A &= \text{boire}, \text{chat}, \text{chien}, \text{eau}, \text{lait}, \text{petit} \\
B &= \text{ADJ}, \text{N}, \text{V}
\end{align}
$$

Les deux séries sont ensuite réunies en une seule :

$$
C = \text{boire}, \text{chat}, \text{chien}, \text{eau}, \text{lait}, \text{petit}, \text{ADJ}, \text{N}, \text{V}
$$

Et, enfin, chaque élément du sac de mots est envoyé à la liste des classificateurs, puis transformé en un vecteur creux de $0$ et de $1$ :

$$
\begin{align}
\vec{BOW_1} = \begin{pmatrix}
    \text{(boire :) } &1 \\
    \text{(chat :) } &0 \\
    \text{(chien :) } &0 \\
    \text{(eau :) } &0 \\
    \text{(lait :) } &0 \\
    \text{(petit :) } &0 \\
    \text{(ADJ :) } &0 \\
    \text{(N :) } &0 \\
    \text{(V :) } &1 \\
\end{pmatrix}
\hspace{2em}
\vec{BOW_2} = \begin{pmatrix}
    \text{(boire :) } &0 \\
    \text{(chat :) } &1 \\
    \text{(chien :) } &0 \\
    \text{(eau :) } &0 \\
    \text{(lait :) } &0 \\
    \text{(petit :) } &0 \\
    \text{(ADJ :) } &0 \\
    \text{(N :) } &1 \\
    \text{(V :) } &0 \\
\end{pmatrix}
\hspace{2em}
\vec{BOW_3} = \begin{pmatrix}
    \text{(boire :) } &0 \\
    \text{(chat :) } &0 \\
    \text{(chien :) } &1 \\
    \text{(eau :) } &0 \\
    \text{(lait :) } &0 \\
    \text{(petit :) } &0 \\
    \text{(ADJ :) } &0 \\
    \text{(N :) } &1 \\
    \text{(V :) } &0 \\
\end{pmatrix}
\end{align}
$$

Les vecteurs étant de longueurs égales, il est désormais pertinent de les normaliser :

In [ ]:
A = [1, 0, 0, 0, 0, 0, 0, 0, 1] # boire, V
B = [0, 1, 0, 0, 0, 0, 0, 1, 0] # chat, N
C = [0, 0, 1, 0, 0, 0, 0, 1, 0] # chien, N

print(
    f'La distance entre "boire" et "chat" est de {distance(a=A, b=B) :.4f}',
    f'La distance entre "boire" et "chien" est de {distance(a=A, b=C) :.4f}',
    f'La distance entre "chat" et "chien" est de {distance(a=B, b=C) :.4f}',
    sep="\n"
)

Les résultats s’interprètent facilement :

- les lemmes *chat* et *chien* sont, dans l’espace vectoriel du corpus, équidistants du lemme *boire* ;
- les lemmes *chat* et *chien* sont, dans l’espace vectoriel du corpus, plus proches l’un de l’autre que du lemme *boire*.

En termes de géométrie, le triangle $ABC$ serait isocèle en $A$ avec :

$$
\begin{align}
    \vec{AB} &= \vec{AC} = 2 \\
    \vec{BC} &= 1,4142
\end{align}
$$

![Distance entre les vecteurs](./images/distance-between-vectors.png)

L’encodage *one-hot* est souvent un point de départ pour nourrir d’autres algorithmes comme dans le cas du *word embedding* avec l’aide de *Word2Vec*, des algorithmes mieux à même d’extraire des caractéristiques des vecteurs pour en dégager une similarité lexicale.

## Évaluer l’importance d’un terme dans un document (TF-IDF)

Nous l’avons vu, certains termes dans un corpus sont plus importants que d’autres pour caractériser un texte par rapport à un autre, et leur importance n’est souvent pas proportionnelle à leur fréquence. De là découle la nécessité de repérer les mots vides de sens, les *stopwords*, pour les retirer du sac de mots (BOW) qui le représente. Pour autant, la méthode BOW se contente d’une mesure fréquentielle sans établir de rapport d’importance entre les termes.

Une autre approche, largement répandue dans le traitement automatique du langage naturel, parvient à inférer, de l’analyse fréquentielle, une certaine valeur d’importance aux termes contenus dans le sac de mots. Cette approche repose sur deux principes : la fréquence du terme (TF) et la fréquence du terme dans le corpus (IDF) qui prêtent une signification à la rareté d’un terme.

Sans parler de robustesse, la justification de cette approche repose sur la loi de Zipf qui prévoit que la fréquence d’un terme dans un texte est liée à son rang dans l’ordre des fréquences : le mot le plus fréquent apparaîtrait dix fois plus souvent que le dixième mot le plus fréquent, cent fois plus que le centième etc. En grande partie pour cette raison, la méthode TF-IDF ne souffre pas de la présence des mots vides dans le sac de mots.

### La fréquence du terme (TF)

De l’anglais *term frequency*, la fréquence du terme établit un rapport entre le nombre d’occurrences d’un mot ($w$) dans un document et le nombre total de mots dans ce document ($n$) :

$$
\text{TF}(w, n) = \frac{w}{n}
$$

Prenons un corpus constitué de trois textes :

```txt
(A) Le petit chat boit du lait. Le lait n’est pas bon pour les chats.
(B) Les petits chiens boivent de l’eau. L’eau irait aussi aux chats.
(C) À partir d’un moment, eau ou lait, ils peuvent bien boire ce qu’ils veulent.
```

La taille en mots du document *A* est de 15, quand elle est de 13 pour le document *B* et de 16 pour le document *C*. Intéressons-nous au mot *lait* ($1$) qui apparaît deux fois dans *A* et une fois dans *C*, mais jamais dans *B*. Ses fréquences seront :

$$
\begin{align}
    \text{TF}_{1, A} &= \frac{2}{15} = 0,1333 \\
    \text{TF}_{1, B} &= \frac{0}{13} = 0 \\
    \text{TF}_{1, C} &= \frac{1}{16} = 0,0625 \\
\end{align}
$$

### La fréquence inverse de document (IDF)

Quand la mesure TF s’attachait au terme dans un document, la mesure IDF (*inverse document frequency*) va s’intéresser à la présence du terme dans le corpus entier selon la relation suivante où $d$ représente le nombre de documents où le terme apparaît et $N$ le nombre total de documents :

$$
\text{IDF}(d, N) = \ln{\frac{N}{d}}
$$

Dans notre exemple, la mesure IDF pour le mot *lait* vaut :

$$
\text{IDF}_1 = \ln{\frac{3}{2}} \approx 0.4055
$$

Le calcul du logarithme permet de pondérer le rapport entre $N$ et $d$ dans la mesure où, lors de l’obtention de TF, le résultat était situé dans un intervalle $[0, 1]$.

### La mesure TF-IDF

Au final, la formule TF-IDF est un produit entre TF et IDF. Pour notre exemple avec le mot *lait* :

$$
\begin{align}
    \text{TFIDF}_{1, A} &= \frac{2}{15} \cdot ln \frac{3}{2} \approx 0,0541 \\
    \text{TFIDF}_{1, B} &= \frac{0}{13} \cdot ln \frac{3}{2} = 0 \\
    \text{TFIDF}_{1, C} &= \frac{1}{16} \cdot ln \frac{3}{2} \approx 0,0253 \\
\end{align}
$$

Au regard du mot *lait*, le document *A* apparaît ainsi comme plus pertinent.

Avec Python, nous avons :

In [ ]:
from math import log as ln

def TF_IDF(*, t: int, w: int, d: int, n: int) -> float:
    """Term frequency–inverse document frequency.

    Keyword arguments:
    t -- term frequency in the document
    w -- number of words in the document
    d -- document frequency
    n -- number of documents
    """

    tf = t / w

    idf = ln(n / d)

    return tf * idf

print(
    f'La mesure TF-IDF du mot "lait" dans le document A est de : {TF_IDF(t=2, w=15, d=2, n=3) :.4f}',
    f'La mesure TF-IDF du mot "lait" dans le document B est de : {TF_IDF(t=0, w=13, d=2, n=3) :.4f}',
    f'La mesure TF-IDF du mot "lait" dans le document C est de : {TF_IDF(t=1, w=16, d=2, n=3) :.4f}',
    sep="\n"
)

### Limitations du modèle

Quoique très largement utilisé pour sa facilité de mise en œuvre en dépit du coût en termes de calcul machine, la mesure TF-IDF souffre principalement de son incapacité à traiter la sémantique du terme.

Gardons par ailleurs à l’esprit que la formule accordera mécaniquement davantage d’importance aux documents très volumineux, aussi faudra-t-il penser à les pénaliser ou bien à ne travailler que sur des corpus équilibrés.

## Opérations sur les vecteurs

La représentation mathématique du monde associe aux objets des propriétés et leur assigne des méthodes, comme la couleur du crayon et la fonction de dessiner. Pour un mot, on pourrait lister parmi ses propriétés son genre ou encore sa catégorie grammaticale, et, parmi ses méthodes, sa capacité à qualifier, à signifier ou à se décliner. La nature des objets matérialise parfois une barrière infrangible, tel qu’il est impossible de conjuguer un nombre ou de diviser des lettres.

Une fois sous forme numérique, les mots, textes ou tokens héritent des opérations arithmétiques réservées aux nombres. Il devient possible de les soustraire, de les multiplier ou encore d’obtenir une moyenne, d’en extraire une racine ou de les comparer entre eux.

S’agissant de vecteurs à *n* dimensions, on retiendra classiquement deux opérations :

- la distance qui les sépare dans l’espace vectoriel du corpus ;
- le cosinus de leur angle pour déterminer leur similarité.

### Calculer la distance entre deux vecteurs

Nous l’avons vu, la distance qui sépare deux vecteurs n’a de sens que dans un espace vectoriel borné. L’exemple du sac de mots d’un corpus permet en effet de situer les mots les uns par rapport aux autres dans cet espace sans ne rien présumer de leur relation dans un autre espace. Retirez ou ajoutez un texte et la mesure varierait.

En plus de cette notion de relativité, rappelons qu’on ne peut comparer que des vecteurs de dimensions égales et qu’un vecteur de *n* dimensions ne peut se concevoir que dans un espace à *n* dimensions. Si deux vecteurs $\vec{A}$ et $\vec{B}$ sont chacun décrits par huit composantes, ils seront considérés dans un espace à huit dimensions.

En règle générale, les mots sont convertis en des vecteurs de dimension très élevée, mais la grande majorité de ces dimensions est établie à 0. On parle alors de vecteur creux.

La représentation d’un vecteur de dimension élevée étant malaisée, une opération courante consiste à le convertir dans une dimension inférieure. En un sens, lorsque l’on supprime les signes de ponctuation et les mots vides de sens du sac de mots d’un corpus, on réduit déjà sa dimensionnalité. Nous aborderons cette approche dans un autre calepin pour ne considérer ici que les opérations nécessaires au calcul de la distance entre deux vecteurs.

#### La norme d’un vecteur

Avant toute chose, un vecteur est décrit par trois caractéristiques : sa norme, sa direction et son sens. Nous ne nous intéresserons qu’à la première.

Prenons un vecteur $\vec{A}$ de dimension 4 :

$$
\vec{A} = \begin{pmatrix}
    3 \\
    12 \\
    9 \\
    0
\end{pmatrix}
$$

Le calcul de sa norme est régi par l’extraction de la racine carrée du produit scalaire avec lui-même :

$$
\| \vec{A} \| = \sqrt{\vec{A} \cdot \vec{A}}
$$

Grâce au développement de la formule, nous retrouvons le théorème de Pythagore :

$$
\begin{align}
\| \vec{A} \| &= \sqrt{(3 \times 3) + (12 \times 12) + (9 \times 9) + (0 \times 0)} \\
\| \vec{A} \| &= \sqrt{3^2 + 12^2 + 9^2 + 0^2} \\
\| \vec{A} \| &= \sqrt{234} \\
\| \vec{A} \| &\approx 15,2971
\end{align}
$$

#### Normaliser un vecteur passant par deux points

Reprenons la formule que nous avons déjà abordée plus haut et qui permet de mesurer la distance entre deux vecteurs $\vec{A}$ et $\vec{B}$ :

$$
\| \vec{AB} \| = \sqrt{(A_1 - B_1)^2 + (A_2 - B_2)^2 \dots (A_n - B_n)^2}
$$

Puis mobilisons deux vecteurs de dimension 3 :

$$
\vec{A} = \begin{pmatrix}
    7 \\
    32 \\
    10
\end{pmatrix}
\hspace{2em}
\vec{B} = \begin{pmatrix}
    11 \\
    4 \\
    8
\end{pmatrix}
$$

Avant de leur appliquer la formule :

$$
\begin{align}
    \| \vec{AB} \| &= \sqrt{(7 - 11)^2 + (32 - 4)^2 + (10 - 8)^2} \\
    \| \vec{AB} \| &= \sqrt{-4^2 + 28^2 + 2^2} \\
    \| \vec{AB} \| &= \sqrt{16 + 784 + 4} \\
    \| \vec{AB} \| &\approx 28,3549
\end{align}
$$

En détail, l’opération consiste à trouver un vecteur $\vec{AB}$ qui passe par $A$ et par $B$ :

$$
\begin{align}
    \vec{AB} &= \begin{pmatrix}
        B_1 - A_1 \\
        B_2 - A_2 \\
        B_3 - A_3
    \end{pmatrix} \\
    \vec{AB} &= \begin{pmatrix}
        11 - 7 \\
        4 - 32 \\
        8 - 10
    \end{pmatrix} \\
    \vec{AB} &= \begin{pmatrix}
        4   \\
        -28 \\
        -2
    \end{pmatrix}
\end{align}
$$

Sa norme s’obtient classiquement par la racine carrée de son produit scalaire avec lui-même :

$$
\begin{align}
    \| \vec{AB} \| &= \sqrt{\vec{AB} \cdot \vec{AB}} \\
    \| \vec{AB} \| &= \sqrt{4^2 + (-28)^2 + (-2)^2} \\
    \| \vec{AB} \| &= \sqrt{16 + 784 + 4} \\
    \| \vec{AB} \| &\approx 28,3549
\end{align}
$$

### Évaluer la similarité de deux vecteurs

#### La similarité cosinus

Lorsque l’on parle de similarité de deux vecteurs en traitement automatique du langage naturel, on désigne la **similarité cosinus**, une mesure par laquelle on détermine le cosinus de leur angle dans un intervalle $[-1 , 1]$, où -1 qualifie deux vecteurs opposés, 0 deux vecteurs indépendants et 1 deux vecteurs colinéaires (il serait possible de tracer une droite pour les relier).

La formule pour définir la similarité cosinus vaut :

$$
\cos \theta = \frac{\vec{A} \cdot \vec{B}}{\| \vec{A} \| \| \vec{B} \|}
$$

Reprenons les vecteurs $\vec{A}$ et $\vec{B}$ décrits plus haut et calculons leur similarité :

$$
\begin{align}
    \cos \theta &= \frac{ (A_1 \cdot B_1) + (A_2 \cdot B_2) + (A_3 \cdot B_3) }{ (\sqrt{\vec{A} \cdot \vec{A}}) \times (\sqrt{\vec{B} \cdot \vec{B}}) } \\
    \cos \theta &= \frac{ (7 \times 11) + (32 \times 4) + (10 \times 8) }{ (\sqrt{7^2 + 32^2 + 10^2}) \times (\sqrt{11^2 + 4^2 + 8^2}) } \\
    \cos \theta &= \frac{77 + 128 + 80}{ 34,2491 \times 14,1774 } \\
    \cos \theta &= \frac{285}{ 485,5632 } \\
    \cos \theta &\approx 0,5869
\end{align}
$$

Selon la nature de $\vec{A}$ et $\vec{B}$, nous pourrions avancer que 0,5869 est la mesure de l’indice de ressemblance entre les mots *A* et *B*, ou entre les documents *A* et *B* etc. Une application pratique serait de résoudre par exemple une tâche de classification en effectuant des regroupements.

#### Vérification dans le triangle quelconque

Considérons à présent $\vec{A}$ et $\vec{B}$ comme des points dans un espace vectoriel reliés à l’origine tel que :

$$
\vec{O} = \begin{pmatrix}
    0 \\
    0 \\
    0
\end{pmatrix}
$$

Nous pouvons estimer la distance de vecteurs passant par $\vec{OA}$, $\vec{OB}$ et $\vec{AB}$ grâce au théorème de Pythagore :

$$
\begin{align}
    \vec{OA} &= \sqrt{(O_1 - A_1)^2 + (O_2 - A_2)^2 + (O_3 - A_3)^2} \\
    &= \sqrt{(0 - 7)^2 + (0 - 32)^2 + (0 - 10)^2} \\
    &= \sqrt{49 + 1024 + 100} \\
    &= \sqrt{1173} \\
    &\approx 34,2491 \\
    \vec{OB} &=\sqrt{(0 - 11)^2 + (0 - 4)^2 + (0 - 8)^2} \\
    &= \sqrt{121 + 16 + 64} \\
    &\approx 14,1774 \\
    \vec{AB} &=\sqrt{(7 - 11)^2 + (32 - 4)^2 + (10 - 8)^2} \\
    &= \sqrt{16 + 784 + 4} \\
    &\approx 28,3549 \\
\end{align}
$$

À partir de là, pour trouver $\cos(O)$, nous pouvons invoquer la loi des cosinus dans un triangle quelconque, de telle manière que :

$$
\cos(c) = \frac{a^2 + b^2 - c^2}{2ab}
$$

Appliquée à notre exemple, la formule nous donne :

$$
\begin{align}
    \cos(O) &= \frac{\vec{OA}^2 + \vec{OB}^2 - \vec{AB}^2}{2 \cdot \vec{OA} \cdot \vec{OB}} \\
    &= \frac{34,2491^2 + 14,1774^2 - 28,3549^2}{2 \cdot 34,2491 \cdot 14,1774} \\
    &= \frac{1173 + 201 - 804}{2 \cdot 485,5631} \\
    &= \frac{570}{971,1262} \\
    &\approx 0,5869
\end{align}
$$

#### Similarité de vecteurs binaires : l’indice de Jaccard

Dans l’exemple précédent, les attributs des vecteurs $\vec{A}$ et $\vec{B}$ étaient représentés par des nombres pouvant prendre leur valeur dans l’ensemble $\mathbb{R}$. Et si les possibilités pour les valeurs se limitaient à *0* et *1*, comme avec un encodage *one-hot* ? Dans ce cas particulier, la similarité peut se calculer avec l’indice de Jaccard :

$$
J = \frac{M_{11}}{M_{01} + M_{10} + M_{11}}
$$

Ou, sachant que *n* est le nombre d’attributs :

$$
J = \frac{M_{11}}{n - M_{00}}
$$

Dans ces formules, les quantités valent :

- $M_{00}$ pour le nombre d’attributs qui valent $0$ dans $\vec{A}$ et $0$ dans $\vec{B}$ ;
- $M_{01}$ pour le nombre d’attributs qui valent $0$ dans $\vec{A}$ et $1$ dans $\vec{B}$ ;
- $M_{10}$ pour le nombre d’attributs qui valent $1$ dans $\vec{A}$ et $0$ dans $\vec{B}$ ;
- $M_{11}$ pour le nombre d’attributs qui valent $1$ dans $\vec{A}$ et $1$ dans $\vec{B}$.

Prenons les deux vecteurs suivants :

$$
\vec{A} = \begin{pmatrix}
    0 \\
    0 \\
    1 \\
    1 \\
    1
\end{pmatrix}
\hspace{2em}
\vec{B} = \begin{pmatrix}
    1 \\
    0 \\
    1 \\
    0 \\
    0
\end{pmatrix}
$$

Après application de la formule, nous obtenons :

$$
\begin{align}
    J &= \frac{1}{1 + 2 + 1} \\
    J &= \frac{1}{4} \\
    J &\approx 0,25
\end{align}
$$